In [1]:
import os, importlib.util
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lime.lime_tabular import LimeTabularExplainer

# 1) Dynamically load your teammate’s module
spec = importlib.util.spec_from_file_location(
    "nbc_mod", os.path.join(os.getcwd(), "nbc.py")
)

In [2]:

nbc_mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(nbc_mod)  

# 2) Load & preprocess
df = pd.read_csv("data/mushroom_dataset.csv")
X, y, encoders = nbc_mod.preprocess_data(df)


In [3]:

# 3) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [4]:

# 4) Train the custom NBC
nbc = nbc_mod.NBC()
nbc.train(X_train, y_train)


In [5]:

# Quick sanity check
print("Custom NBC test accuracy:",
      np.mean(nbc.predict(X_test) == y_test))


Custom NBC test accuracy: 0.9458572600492207


In [6]:
# ------------------------------------------------------------------
#  LIME-on-k–medoids  pipeline  (one‑hot version)
# ------------------------------------------------------------------
import pandas as pd, numpy as np, joblib, json
from sklearn.preprocessing import OneHotEncoder
from sklearn_extra.cluster import KMedoids          # pip install scikit-learn-extra
from lime.lime_tabular import LimeTabularExplainer
from collections import defaultdict, Counter
import sklearn
from sklearn.preprocessing import OneHotEncoder
from packaging import version


# ---------- 1. one‑hot encode the original df ---------------------
df = pd.read_csv("data/mushroom_dataset.csv")
y_full = df["class"].map({"e": 0, "p": 1}).to_numpy()
X_raw  = df.drop(columns="class")


if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
else:
    ohe = OneHotEncoder(sparse=False,       handle_unknown="ignore")

X_onehot = ohe.fit_transform(X_raw)
onehot_feature_names = ohe.get_feature_names_out(X_raw.columns)


In [7]:

# ---------- 2. train/test split (same split as your NBC) ----------
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te, idx_tr, idx_te = train_test_split(
    X_onehot, y_full, np.arange(len(y_full)),
    test_size=0.3, random_state=42, stratify=y_full
)


In [8]:

# ---------------------------------------------------------------
# 1.  LOAD & SPLIT DATA  (label‑encoded + one‑hot)
# ---------------------------------------------------------------
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import Bunch
from nbc import NBC           # your custom Naive Bayes class

df = pd.read_csv("data/mushroom_dataset.csv")
X_raw, y_raw = df.drop(columns="class"), df["class"]


In [9]:

# --- label‑encode every column (NBC needs this) ---
X_le = X_raw.copy()
encoders = {}
for col in X_le.columns:
    le = LabelEncoder().fit(X_le[col])
    X_le[col] = le.transform(X_le[col])
    encoders[col] = le

y = y_raw.values



In [10]:
# --- one‑hot encode for distance / LIME ---
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
X_oh = ohe.fit_transform(X_raw)
onehot_names = ohe.get_feature_names_out(X_raw.columns)


In [11]:

# one single split so rows align
X_le_tr, X_le_te, X_oh_tr, X_oh_te, y_tr, y_te = train_test_split(
    X_le, X_oh, y, test_size=0.3, random_state=42, stratify=y
)


In [13]:
# pick 40 medoids on the ONE‑HOT test set
from sklearn_extra.cluster import KMedoids

kmed   = KMedoids(n_clusters=40, metric="manhattan", random_state=0)
kmed.fit(X_oh_te)
rep_idx = kmed.medoid_indices_


In [14]:
# ───────────────────────────────────────────────────────────────────────────────
# Cell 13: LIME in label‑encoded space for NBC (feature=value style)
# ───────────────────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
from lime.lime_tabular  import LimeTabularExplainer
from collections        import Counter, defaultdict
from utils import text_utils

# 1) We already did:
#    X_le_tr, X_le_te, X_oh_tr, X_oh_te, y_tr, y_te = train_test_split(...)
#    encoders = {col: LabelEncoder()… }  # from Cell 8
#    ohe      = OneHotEncoder()…         # from Cell 9
#    nbc.train(X_le_tr, y_tr)            # earlier cell

# 2) Pick 40 medoids on the ONE‑HOT test set (Cell 12)
#    (we reuse its rep_idx)
# rep_idx = kmed.medoid_indices_

# 3) Build label‑encoded arrays
bg_le  = X_le_tr.values                  # train background in label space
med_le = X_le_te.values[rep_idx]         # the 40 test medoids in label space

# 4) Tell LIME these d are all categorical features
d = bg_le.shape[1]
cat_feats = list(range(d))
cat_names  = {
    i: encoders[col].classes_.tolist()
    for i, col in enumerate(X_le_tr.columns)
}

# 5) Wrap your NBC so it returns [P(edible), P(poisonous)] on label‑encoded rows
def model_proba_le(arr_le):
    arr = np.atleast_2d(arr_le)
    prob_e, prob_p = [], []
    for sample in arr:
        # reconstruct a pandas Series for your predict_one
        entry = pd.Series({col: sample[i]
                           for i, col in enumerate(X_le_tr.columns)})
        # compute unnormalised class‑likelihoods just as in model_proba
        unnorm = {}
        for cls in nbc.output_classes:
            p0 = nbc.output_class_probs[cls]
            for ft, val in entry.items():
                p0 *= nbc.per_class_feature_probs[cls][ft][val]
            unnorm[cls] = p0
        # normalise
        total = sum(unnorm.values())
        prob_e.append(unnorm['e'] / total)
        prob_p.append(unnorm['p'] / total)
    return np.column_stack([prob_e, prob_p])

# 6) Run LIME on each medoid × 3 seeds, aggregate top‑5 hits
agg_freq, agg_wt = Counter(), defaultdict(float)
for seed in (0, 1, 2):
    explainer = LimeTabularExplainer(
        training_data        = bg_le,
        feature_names        = list(X_le_tr.columns),
        class_names          = ["edible","poisonous"],
        categorical_features = cat_feats,
        categorical_names    = cat_names,
        mode                 = "classification",
        random_state         = seed
    )
    for row in med_le:
        exp = explainer.explain_instance(
            row,
            model_proba_le,
            num_features=5
        )
        for feat, wt in exp.as_list(label=1):
            agg_freq[feat] += 1
            agg_wt[feat]   += wt

# 7) Build your summary table just like before
k_total = len(rep_idx) * 3  # 40 medoids × 3 seeds
rows = []
for feat in agg_freq:
    rows.append({
        "feature":            feat,
        "pct_in_top5":       100 * agg_freq[feat]   / k_total,
        "mean_signed_weight": agg_wt[feat]         / agg_freq[feat]
    })

summary_df = pd.DataFrame(rows).sort_values(
    ["pct_in_top5", "mean_signed_weight"],
    ascending=False
)

print(summary_df)

text_utils.ensure_directory_exists("eval/lime_results")
summary_df.to_csv("eval/lime_results/lime_nbc_summary_label_encoded.csv", index=False)


                       feature  pct_in_top5  mean_signed_weight
1                  ring-type=p    67.500000           -0.154501
4   stalk-surface-above-ring=s    66.666667           -0.120916
0                       odor=n    62.500000           -0.379259
3                  gill-size=b    52.500000           -0.122882
5                 population=v    35.833333            0.112640
6   stalk-surface-below-ring=s    21.666667           -0.111141
9                       odor=f    20.000000            0.389031
24  stalk-surface-above-ring=k    17.500000            0.150217
21  stalk-surface-below-ring=k    15.833333            0.145587
10                gill-color=b    15.000000            0.309639
22                 ring-type=l    15.000000            0.274615
8                    bruises=t    15.000000           -0.108866
23         spore-print-color=h    14.166667            0.166090
13                 gill-size=n    14.166667            0.123816
20         spore-print-color=n    11.666

In [ ]:

# ---------------------------------------------------------------
# 2.  TRAIN NBC ON LABEL‑ENCODED TRAIN DATA
# ---------------------------------------------------------------
nbc = NBC()
nbc.train(X_le_tr, y_tr)          # custom implementation



In [ ]:
# ---------------------------------------------------------------
# 3.  PICK k MEDOIDS (or K‑means) ON ONE‑HOT TEST MATRIX
# ---------------------------------------------------------------
kmed    = KMedoids(n_clusters=40, metric="manhattan", random_state=0)
kmed.fit(X_oh_te)
rep_idx = kmed.medoid_indices_

# (3) Build label‑encoded background & medoids
bg_le   = X_le_tr.values      # shape (n_train, d)
med_le  = X_le_te.values[rep_idx]  # shape (40, d)

# (4) Setup LIME on label‑encoded data
d = bg_le.shape[1]
cat_feats = list(range(d))
cat_names = {
    i: label_encoders[col].classes_.tolist()
    for i, col in enumerate(X_le_tr.columns)
}

In [ ]:
# get corresponding rows in each representation
X_oh_reps = X_oh_te[rep_idx_te]
X_le_reps = X_le_te.iloc[rep_idx_te]



In [ ]:
# ---------------------------------------------------------------
# 4.  LIME ON EACH REPRESENTATIVE POINT (3 seeds each)
# ---------------------------------------------------------------
from lime.lime_tabular import LimeTabularExplainer
from collections import Counter, defaultdict

explainer = LimeTabularExplainer(
    training_data=X_oh_tr,
    feature_names=onehot_names,
    class_names=["edible", "poisonous"],
    discretize_continuous=False
)
def make_explainer(seed):
    return LimeTabularExplainer(
        X_oh_tr,
        feature_names=onehot_names,
        class_names=["edible", "poisonous"],
        discretize_continuous=False,
        random_state=seed
    )



In [ ]:
def model_proba(X_oh_batch):
    """
    Parameters
    ----------
    X_oh_batch : ndarray (n_samples, n_onehot_features)
        0/1 one‑hot rows supplied by LIME.

    Returns
    -------
    probs : ndarray (n_samples, 2)
        Columns: P(edible) , P(poisonous)
    """
    # --- 1. one‑hot  ➜  original string categories -------------
    cat_matrix = ohe.inverse_transform(X_oh_batch)   # shape (n_samples, 117)

    # --- 2. string categories  ➜  label‑encoded ints -----------
    #     vectorised column‑by‑column
    le_matrix = np.empty_like(cat_matrix, dtype=int)
    for j, col in enumerate(X_raw.columns):
        le_matrix[:, j] = encoders[col].transform(cat_matrix[:, j])

    # --- 3. NBC probability for each row -----------------------
    prob_e = []
    prob_p = []
    for row in le_matrix:
        # row is a 1‑D int array; wrap in dict {feature: value}
        entry = dict(zip(X_raw.columns, row))
        # use your NBC's predict_one, which returns (class, prob)
        pred_class, pred_prob = nbc.predict_one(entry)

        # compute the *unnormalised* probs for each class
        unnorm = {}
        for cls in nbc.output_classes:
            p = nbc.output_class_probs[cls]
            for ft, val in entry.items():
                p *= nbc.per_class_feature_probs[cls][ft][val]
            unnorm[cls] = p
        total = unnorm['e'] + unnorm['p']
        prob_e.append(unnorm['e'] / total)
        prob_p.append(unnorm['p'] / total)

    return np.column_stack([prob_e, prob_p])


In [ ]:
agg_freq, agg_wt = Counter(), defaultdict(float)
seeds = (0, 1, 2)

# (6) Loop seeds & medoids
for seed in seeds:
    explainer = LimeTabularExplainer(
        training_data        = bg_le,
        feature_names        = list(X_le_tr.columns),
        class_names          = ["edible","poisonous"],
        categorical_features = cat_feats,
        categorical_names    = cat_names,
        mode                 = "classification",
        random_state         = seed

In [ ]:


 for row in med_le:
        exp = explainer.explain_instance(
            row, 
            lambda arr: np.array([nbc.predict_one(pd.Series(x))[1] for x in arr]), 
            num_features=5
        )
        for feat, wt in exp.as_list(label=1):
            agg_freq[feat] += 1
            agg_wt[feat]   += wt






In [ ]:


# ---------------------------------------------------------------
# 5.  SUMMARY CSV
# ---------------------------------------------------------------
k_total = k * len(seeds)
rows = [
    {
        "feature": f,
        "pct_in_top5": 100 * agg_freq[f] / k_total,
        "mean_signed_weight": agg_weight[f] / agg_freq[f],
    }
    for f in agg_freq
]
summary_df = pd.DataFrame(rows).sort_values(
    ["pct_in_top5", "mean_signed_weight"], ascending=False
)

summary_df.to_csv("eval/lime_results/lime_nbc_summary1.csv", index=False)
print("sweet it worked: wrote lime_nbc_summary.csv with", len(summary_df), "rows")


odor_f	100 %	+0.106	“Odor = foul” is in every explanation and increases the NBC’s confidence in the predicted class (almost always poisonous). It’s the model’s strongest universal cue.
gill‑color_b	100 %	+0.088	Having brown gills consistently supports the prediction (likely poisonous).
ring‑type_l	100 %	+0.053	Large ring type also pushes probability toward the predicted class.
odor_n	100 %	‑0.132	“Odor = none” appears everywhere but with a negative effect – it pulls the model away from the current prediction. That usually means the model uses odor = none as evidence against poison.
ring‑type_p	98 %	‑0.050	Partial ring is almost everywhere and slightly decreases confidence; perhaps NBC associates partial rings with edible mushrooms.
spore‑print‑color_h	1.7 %	+0.041	“Spore‑print = chocolate” (value h) barely shows up (only 2 runs out of 120) so it’s not a globally important cue, but when it does appear it nudges the probability up.